## Features Radiomiche

In [2]:
import os
import radiomics
from radiomics import featureextractor
import numpy as np
import nrrd

# Percorsi delle cartelle
pazienti_dir = 'C:\\Users\\bsbar\\Desktop\\pazienti_nrrd'
roi_dir = 'C:\\Users\\bsbar\\Desktop\\Tesi\\ROI'

cartelle_da_saltare = ['19', '40', '88', '150', '193', '107']

# Lista per i vettori delle features e i nomi dei pazienti
feature_vectors = []
patient_names = []
dimension_mismatch = []

# Imposta l'estrattore con le configurazioni di default
estrattore = featureextractor.RadiomicsFeatureExtractor()

# Funzione per trovare il file .nrrd in una cartella
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None

# Funzione per verificare le dimensioni delle immagini
def verifica_dimensioni(immagine_file, maschera_file):
    immagine, _ = nrrd.read(immagine_file)
    maschera, _ = nrrd.read(maschera_file)
    return immagine.shape == maschera.shape

# Itera su tutti i pazienti
for paziente in sorted(os.listdir(pazienti_dir)):
    print(paziente)
    if paziente in cartelle_da_saltare:
        print(f"Saltata la cartella del paziente {paziente}")
        continue  # Salta questa iterazione e passa al paziente successivo
    
    # Verifica se la cartella esiste anche per le ROI
    roi_paziente_dir = os.path.join(roi_dir, paziente)
    paziente_dir = os.path.join(pazienti_dir, paziente)

    if os.path.isdir(roi_paziente_dir):
        # Trova i file immagine e maschera nella rispettiva cartella
        immagine_file = trova_nrrd_file(paziente_dir)
        maschera_file = trova_nrrd_file(roi_paziente_dir)
     
        if immagine_file and maschera_file:
            # Verifica se le dimensioni delle immagini e delle ROI corrispondono
            if verifica_dimensioni(immagine_file, maschera_file):
                # Aggiungi il nome del paziente alla lista
                patient_names.append(paziente)

                # Estrai le features radiomiche
                results = estrattore.execute(immagine_file, maschera_file)

                # Trasforma i risultati in un vettore di features
                feature_vector = np.array([value.item() if isinstance(value, np.ndarray) else value 
                           for key, value in results.items() 
                           if isinstance(value, (float, int, np.ndarray))])
                
                # Aggiungi il vettore delle features alla lista
                feature_vectors.append(feature_vector)
                
            else:
                # Aggiungi il nome del paziente e un messaggio di errore alla lista delle dimensioni non corrispondenti
                dimension_mismatch.append(paziente)

# Trasforma la lista di vettori in un array numpy
feature_matrix = np.array(feature_vectors)

# Ora hai:
# - 'feature_matrix': una matrice dove ogni riga è il vettore di features di un paziente
# - 'patient_names': una lista dei nomi dei pazienti corrispondente
# - 'dimension_mismatch': una lista dei nomi dei pazienti con dimensioni non corrispondenti

100


KeyboardInterrupt: 

In [26]:
import csv

# Ordina i pazienti in base al loro nome numerico
patient_names_sorted, feature_vectors_sorted = zip(*sorted(zip(patient_names, feature_vectors), key=lambda x: int(x[0])))
feature_names = [key for key, value in results.items() if isinstance(value, (float, int, np.ndarray))]


# Salva le features e i nomi dei pazienti in un file CSV
output_csv = 'features_radiomiche.csv'
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=';')  # Usa il delimitatore punto e virgola

    # Scrivi l'intestazione: la prima colonna è "Paziente" e le altre colonne rappresentano le features
    writer.writerow(['Paziente'] + feature_names)

    # Scrivi i dati: il nome del paziente e le sue features
    for name, features in zip(patient_names_sorted, feature_vectors_sorted):
        writer.writerow([name] + list(features))

print(f'File CSV salvato come {output_csv}')

File CSV salvato come features_radiomiche.csv


In [36]:
import radiomics
from radiomics import featureextractor, getFeatureClasses

# Ottieni le classi di feature disponibili
feature_classes = getFeatureClasses()
print("Feature classes disponibili:")
for feature_class_name in feature_classes:
    print(feature_class_name)

Feature classes disponibili:
firstorder
glcm
gldm
glrlm
glszm
ngtdm
shape
shape2D


## Wavelet

In [9]:
import os
import radiomics
from radiomics import featureextractor
import numpy as np
import nrrd

# Percorsi delle cartelle
pazienti_dir = 'C:\\Users\\bsbar\\Desktop\\pazienti_nrrd'
roi_dir = 'C:\\Users\\bsbar\\Desktop\\Tesi\\ROI'

cartelle_da_saltare = ['19', '40', '88', '150', '193', '107']

# Lista per i vettori delle features e i nomi dei pazienti
feature_vectors = []
patient_names = []
dimension_mismatch = []

# Configurazione dell'estrattore radiomico
settings = {
    'binWidth': 25,
    'resampledPixelSpacing': None,
    'interpolator': 'sitkBSpline',
    'enableCExtensions': True
}

# Seleziona esplicitamente quali feature estrarre
enabled_features = {
    'firstorder': [],
    'shape': [],
    'glcm': [],
    'gldm': [],
    'glrlm': [],
    'glszm': [],
    'ngtdm': [],
}

# Aggiungi i filtri, inclusa la wavelet
enabled_image_types = {
    'Original': {},  # Usa l'immagine originale
    'Wavelet': {},   # Aggiungi il filtro wavelet
    'LoG': {}        # Aggiungi anche il filtro LoG se vuoi testarlo
}

# Inizializza l'estrattore con queste configurazioni
estrattore = featureextractor.RadiomicsFeatureExtractor(
    settings, enabled_image_types, enabled_features
)

# Funzione per trovare il file .nrrd in una cartella
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None

# Funzione per verificare le dimensioni delle immagini
def verifica_dimensioni(immagine_file, maschera_file):
    immagine, _ = nrrd.read(immagine_file)
    maschera, _ = nrrd.read(maschera_file)
    return immagine.shape == maschera.shape

# Itera su tutti i pazienti
for paziente in sorted(os.listdir(pazienti_dir)):
    print(paziente)
    if paziente in cartelle_da_saltare:
        print(f"Saltata la cartella del paziente {paziente}")
        continue  # Salta questa iterazione e passa al paziente successivo
    
    # Verifica se la cartella esiste anche per le ROI
    roi_paziente_dir = os.path.join(roi_dir, paziente)
    paziente_dir = os.path.join(pazienti_dir, paziente)

    if os.path.isdir(roi_paziente_dir):
        # Trova i file immagine e maschera nella rispettiva cartella
        immagine_file = trova_nrrd_file(paziente_dir)
        maschera_file = trova_nrrd_file(roi_paziente_dir)
     
        if immagine_file and maschera_file:
            # Verifica se le dimensioni delle immagini e delle ROI corrispondono
            if verifica_dimensioni(immagine_file, maschera_file):
                # Aggiungi il nome del paziente alla lista
                patient_names.append(paziente)

                # Estrai le features radiomiche, incluse quelle wavelet e LoG, direttamente dall'immagine e dalla maschera
                results = estrattore.execute(immagine_file, maschera_file)

                # Trasforma i risultati in un vettore di features
                feature_vector = np.array([value.item() if isinstance(value, np.ndarray) else value 
                           for key, value in results.items() 
                           if isinstance(value, (float, int, np.ndarray))])
                
                # Aggiungi il vettore delle features alla lista
                feature_vectors.append(feature_vector)
                
            else:
                # Aggiungi il nome del paziente e un messaggio di errore alla lista delle dimensioni non corrispondenti
                dimension_mismatch.append(paziente)

# Trasforma la lista di vettori in un array numpy
feature_matrix = np.array(feature_vectors)

# Ora hai:
# - 'feature_matrix': una matrice dove ogni riga è il vettore di features di un paziente
# - 'patient_names': una lista dei nomi dei pazienti corrispondente
# - 'dimension_mismatch': una lista dei nomi dei pazienti con dimensioni non corrispondenti

100


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


101


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


102


...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


96


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


98


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


## Features Area

In [4]:
import os
import numpy as np
import nrrd
import pandas as pd
from radiomics import featureextractor
from skimage.util import view_as_blocks
import SimpleITK as sitk

# Percorsi delle cartelle
pazienti_dir = 'C:\\Users\\bsbar\\Desktop\\pazienti_nrrd'
roi_dir = 'C:\\Users\\bsbar\\Desktop\\Tesi\\ROI'

# Cartelle da saltare
cartelle_da_saltare = ['19', '40', '88', '150', '193']

# Lista per i vettori delle features e i nomi dei pazienti
feature_vectors = []
patient_names = []
blocchi_numeri = []
dimension_mismatch = []

# Imposta l'estrattore radiomico
estrattore = featureextractor.RadiomicsFeatureExtractor()

# Funzione per trovare il file .nrrd in una cartella
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None

# Funzione per caricare il volume 3D delle immagini
def carica_volume_paziente(cartella_paziente):
    sezioni = []  # Lista per conservare le singole sezioni 2D
    for file in sorted(os.listdir(cartella_paziente)):
        if file.endswith('.nrrd'):
            # Leggi la sezione 2D
            immagine, _ = nrrd.read(os.path.join(cartella_paziente, file))
            sezioni.append(immagine)  # Aggiungi la sezione alla lista
    # Stack delle sezioni per formare un volume 3D (asse delle sezioni = asse z)
    volume_3d = np.stack(sezioni, axis=-1)
    return volume_3d

# Funzione per suddividere il volume in blocchi 3D
def suddividi_volume(volume_3d, dimensione_blocco=(16, 16, 16)):
    # Verifica se le dimensioni del volume sono divisibili esattamente per il blocco
    shape_volume = np.array(volume_3d.shape)
    if np.any(shape_volume % np.array(dimensione_blocco) != 0):
        raise ValueError("Le dimensioni del volume non sono divisibili esattamente per il blocco 3D.")
    
    # Usa view_as_blocks per suddividere il volume in blocchi 3D
    blocchi = view_as_blocks(volume_3d, block_shape=dimensione_blocco)
    
    # Riorganizza i blocchi in una lista di blocchi singoli
    blocchi_risagomati = blocchi.reshape(-1, *dimensione_blocco)
    return blocchi_risagomati

# Funzione per convertire un array NumPy in un oggetto SimpleITK
def numpy_to_sitk(volume_numpy):
    return sitk.GetImageFromArray(volume_numpy)

# Funzione per verificare se la maschera ha valori non nulli
def verifica_maschera_valida(maschera):
    return np.any(maschera > 0)

# Funzione per estrarre le features da ciascun blocco
def estrai_features_blocchi(blocchi, maschere):
    feature_vectors = []
    blocchi_successo = []
    feature_names = None
    
    for i, (blocco, maschera) in enumerate(zip(blocchi, maschere)):
        if not verifica_maschera_valida(maschera):
            continue
        
        if blocco.ndim < 3 or maschera.ndim < 3:
            continue
        
        sitk_blocco = numpy_to_sitk(blocco)
        sitk_maschera = numpy_to_sitk(maschera)
        
        try:
            results = estrattore.execute(sitk_blocco, sitk_maschera)
            if feature_names is None:
                feature_names = list(results.keys())
                
            feature_vector = [results.get(key, np.nan) for key in feature_names]
            feature_vectors.append(feature_vector)
            blocchi_successo.append(i)
        except ValueError as e:
            print(f"Errore durante l'estrazione delle features per il blocco {i}: {e}")
            continue

    return feature_vectors, blocchi_successo, feature_names

def riduci_slice(volume_3d, target_slices=512):
    slice_correnti = volume_3d.shape[-1]
    
    if slice_correnti < target_slices:
        slice_da_aggiungere = target_slices - slice_correnti
        slice_inizio = slice_da_aggiungere // 2
        slice_fine = slice_da_aggiungere - slice_inizio
        padding = ((0, 0), (0, 0), (slice_inizio, slice_fine))
        volume_pad = np.pad(volume_3d, pad_width=padding, mode='constant', constant_values=0)
        return volume_pad

    elif slice_correnti > target_slices:
        slice_da_rimuovere = slice_correnti - target_slices
        slice_inizio = slice_da_rimuovere // 2
        slice_fine = slice_da_rimuovere - slice_inizio
        volume_ridotto = volume_3d[:, :, slice_inizio:slice_correnti-slice_fine]
        return volume_ridotto

    else:
        return volume_3d

# Funzione per suddividere sia la placca che la maschera
def suddividi_placca_maschera(volume_3d, maschera, dimensione_blocco=(32, 32, 32)):
    # Applica la maschera per estrarre solo la parte della placca
    placca = np.where(maschera > 0, volume_3d, 0)
    
    # Verifica se le dimensioni della placca e della maschera sono divisibili esattamente per il blocco
    shape_placca = np.array(placca.shape)
    shape_maschera = np.array(maschera.shape)
    if np.any(shape_placca % np.array(dimensione_blocco) != 0) or np.any(shape_maschera % np.array(dimensione_blocco) != 0):
        raise ValueError("Le dimensioni della placca o della maschera non sono divisibili esattamente per il blocco 3D.")
    
    # Suddividi in blocchi sia la placca che la maschera
    blocchi_placca = view_as_blocks(placca, block_shape=dimensione_blocco)
    blocchi_maschera = view_as_blocks(maschera, block_shape=dimensione_blocco)
    
    # Riorganizza i blocchi in liste di blocchi singoli
    blocchi_placca_risagomati = blocchi_placca.reshape(-1, *dimensione_blocco)
    blocchi_maschera_risagomati = blocchi_maschera.reshape(-1, *dimensione_blocco)
    
    return blocchi_placca_risagomati, blocchi_maschera_risagomati

# Itera su tutti i pazienti
for paziente in sorted(os.listdir(pazienti_dir)):
    if paziente in cartelle_da_saltare:
        print(f"Saltando paziente: {paziente}")
        continue

    print(f"Processando paziente: {paziente}")
    
    immagine_file = trova_nrrd_file(os.path.join(pazienti_dir, paziente))
    maschera_file = trova_nrrd_file(os.path.join(roi_dir, paziente))
    
    if immagine_file and maschera_file:
        volume_placca = carica_volume_paziente(os.path.join(pazienti_dir, paziente))
        volume_maschera = carica_volume_paziente(os.path.join(roi_dir, paziente))
        
        if volume_placca.shape == volume_maschera.shape:
            if len(volume_placca.shape) == 4:
                volume_placca = np.squeeze(volume_placca, axis = -1)
                volume_maschera = np.squeeze(volume_maschera, axis = -1)

            if volume_placca.shape[2] != 512:
                volume_placca = riduci_slice(volume_placca)
                volume_maschera = riduci_slice(volume_maschera)

            # Suddividi sia la placca che la maschera
            blocchi_placca, blocchi_maschera = suddividi_placca_maschera(volume_placca, volume_maschera, dimensione_blocco=(16, 16, 16))
            
            # Estrai le features dai blocchi di placca e dalle corrispondenti maschere
            features_blocchi, blocchi_successo, feature_names = estrai_features_blocchi(blocchi_placca, blocchi_maschera)
            
            for blocco, features in zip(blocchi_successo, features_blocchi):
                feature_vectors.append(features)
                patient_names.append(paziente)
                blocchi_numeri.append(blocco)
        else:
            print(f"Dimensioni non corrispondenti per il paziente {paziente}")
            dimension_mismatch.append(paziente)
    else:
        print(f"File mancanti per il paziente {paziente}")

# Crea un DataFrame e salva come CSV
df = pd.DataFrame(feature_vectors, columns=feature_names)
df.insert(0, 'paziente', patient_names)
df.insert(1, 'blocco', blocchi_numeri)

# Salva il DataFrame in un file CSV
df.to_csv('features_radiomiche_blocchi.csv', index=False)

Processando paziente: 100


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Errore durante l'estrazione delle features per il blocco 13708: mask only contains 1 segmented voxel! Cannot extract features for a single voxel.
Processando paziente: 101


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Errore durante l'estrazione delle features per il blocco 21996: mask has too few dimensions (number of dimensions 1, minimum required 2)


...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Processando paziente: 96


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Processando paziente: 98


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

In [6]:
import pandas as pd

# Carica il file CSV
file_path = 'features_radiomiche_blocchi.csv'
df = pd.read_csv(file_path)

# Stampa tutti i nomi delle colonne
print("Nomi delle colonne:")
print(df.columns.tolist())

# Specifica i nomi delle colonne da eliminare
colonne_da_eliminare = ['diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size', 'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass']  # Sostituisci con i nomi reali delle colonne da eliminare

# Elimina le colonne specificate
df = df.drop(columns=colonne_da_eliminare)

# Ordina il DataFrame in base alla prima colonna per numero in ordine crescente
df_sorted = df.sort_values(by=df.columns[0])

# Salva il DataFrame ordinato in un nuovo file CSV
df_sorted.to_csv('features_blocchi_nuove_16.csv', index=False)


Nomi delle colonne:
['paziente', 'blocco', 'diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size', 'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass', 'original_shape_Elongation', 'original_shape_Flatness', 'original_shape_LeastAxisLength', 'original_shape_MajorAxisLength', 'origin

## Salvataggio

In [25]:
import csv

# Ordina i pazienti in base al loro nome numerico
patient_names_sorted, feature_vectors_sorted = zip(*sorted(zip(patient_names, feature_vectors), key=lambda x: int(x[0])))
feature_names = [key for key, value in results.items() if isinstance(value, (float, int, np.ndarray))]


# Salva le features e i nomi dei pazienti in un file CSV
output_csv = 'features_radiomiche_blocchi.csv'
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=';')  # Usa il delimitatore punto e virgola

    # Scrivi l'intestazione: la prima colonna è "Paziente" e le altre colonne rappresentano le features
    writer.writerow(['Paziente'] + feature_names)

    # Scrivi i dati: il nome del paziente e le sue features
    for name, features in zip(patient_names_sorted, feature_vectors_sorted):
        writer.writerow([name] + list(features))

print(f'File CSV salvato come {output_csv}')

NameError: name 'results' is not defined